# Segmenting and Clustering Neighborhoods in Toronto

### Second part: Adding geographical coordinates

In [10]:
# Web scraping (part one)

#pip install lxml
#pip install bs4
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable'})

# Columns from the Wikipedia table
col_name = My_table.findAll("th")
columnas = []
for x in col_name:
    columnas.append(x.get_text())
columnas = [x.strip() for x in columnas if str(x)]

# Data from table
data = My_table.findAll("td")

info = []
for x in data:
    info.append(x.get_text())
info = [x.strip() for x in info if str(x)]

postal_code = [info[x] for x in range(0,len(info),3)]
borough = [info[x+1] for x in range(0,len(info)-1,3)]
neighborhood = [info[x+2] for x in range(0,len(info)-2,3)]

# Dataframe 
temp_df = pd.DataFrame([postal_code, borough, neighborhood]).T
temp_df.columns = ["PostalCode","Borough", "Neighborhood"]

temp_df = temp_df[temp_df["Borough"] != "Not assigned"]
temp_df.reset_index(inplace = True, drop = True)

temp_df['Borough'] = temp_df['Borough'].replace('\s+', ' ', regex = True)
for x in range(len(temp_df.index)):
    if temp_df["Neighborhood"][x] == "":
        temp_df["Neighborhood"][x] = temp_df["Borough"][x]
    else:
        pass

temp_df['Neighborhood'] = temp_df['Neighborhood'].replace('\s+', ' ', regex = True)
temp_df['Neighborhood'] = temp_df['Neighborhood'].str.replace(' /',',')

# temp_df.head()
# temp_df.shape

In [11]:
# Geographical coordinates (second part)

coordinate = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")

temp_df = temp_df.merge(
    coordinate, 
    how = "inner", 
    left_on = "PostalCode", 
    right_on = "Postal Code",
    suffixes=('_master', '_using')
    )

df = temp_df.drop(labels = 'Postal Code', axis = 1)

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
